In [1]:
import morePVs_output as opm
import morePVs as mpv
import os
import en_utilities as um
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as mlines



Plotting NPV
-------------------



In [2]:

project='EN1_rerun3'
study_name='value11b'
base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_4\\studies'
op = opm.Output(base_path = base_path,
                          project = project,
                          study_name = study_name)

short_name = 'value11b'

In [3]:
self = op
df = self.data.copy()
df_in = self.study_parameters.copy()


In [4]:

df = df.merge(df_in,left_index = True, right_index=True)
for c in df.columns:
    if '_x' in c:
        nc = c[:-2]
        df.rename(columns={c:nc}, inplace=True)


In [5]:
# Set up constants
sites = ['A','E',  'D','B', 'H', 'I', 'G', 'C',  'J', 'F' ]
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
ordered_sites = dict(zip(sites,values))
tag_list = ['a208_f12_cp34',
 'a161_f7_cp38',
 'a138_f9_cp44',
 'a104_f8_cp57',
 'a52_f3_cp26',
 'a48_f4_cp09',
 'a44_f4_cp17',
 'a34_f4_cp33',
 'a26_f4_cp43',
 'a20_f5_cp36']
tag_dict=dict(zip(sites, tag_list))
tag_sorter= dict(zip(tag_list,values))


# Plotting set-up parameters
cmap = mpl.cm.tab10_r
alpha=1
sites_grouped={'all': sites,
              'group1': [s for s in sites if s in ['A','B','C','D','E']],
              'group2': [s for s in sites if s in['F', 'G','H','I','J']]}

a_terms =[a for a in [5,10,15,20,25] if a in df.a_term.tolist()]
dashList = [(2,8),(3,5),(10,0),(3,2,10,2),(5,2,20,2)] 
markers =['+','o','d','s','x']
term_dashes = dict(zip(a_terms,dashList))
term_markers = dict(zip(a_terms,markers))

# get pv sizes for maximum pv systems
maximums={}
pv_ref_file='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\reference\\capex_pv_lookup.csv'
pv_ref=pd.read_csv(pv_ref_file)
pv_ref = pv_ref.set_index('pv_cap_id')
for i in pv_ref.index:
    if 'max' in i and 'site' in i:
        site = um.find_between(i,'_','_')
        maximums[site] = pv_ref.loc[i,'kW']
en_capexes= [c for c in ['capex_low','capex_med','capex_high']if c in df.en_capex_id.tolist()]
#parents = [p for p in ['EA310_12c_FIT8', 'EA310_12c_FIT12', 'EA310_12c',\
#           'EA310_10c_FIT8', 'EA310_10c_FIT12', 'EA310_10c',\
#         'EA310_8c_FIT8', 'EA310_8c_FIT12','EA310_8c'] if p in df.parent.tolist()]
parents_retail = list(set([p[6:] for p in df.parent.tolist() if p != 'TIDNULL']))

In [6]:
# Organise labels (no longer used for labels- now use site_tags)
df.loc[:,'site'] = df.loc[:,'load_folder'].apply(lambda x : x[-1])

floors = {'A':12,'E':7, 'D':9,'B':8,'H':3,'I':4,'G':4,'C':4,'J':4,'F':5}
numhouses={'A':208,'E':161, 'D':138,'B':104,'H':52,'I':48,'G':44,'C':34,'J':26,'F':20}
labels ={}
for s, f in floors.items():
    u = numhouses[s]
    labels[s] = s +'('+str(u)+'/'+str(f)+')'

df.loc[:, 'label'] = df.loc[:, 'site'].apply(lambda x: labels[x])




In [7]:
# get kWp / unit , total $ / unit  and % bau for each scenario
df['pv_filename'].fillna('zero',inplace=True)
for s in df.index:
    site = df.loc[s,'load_folder'][-1]
    a_term = df.loc[s,'a_term']
    df.loc[s,'site']=site
    if df.loc[s,'pv_filename'] == 'zero':
        df.loc[s,'kwp_unit'] =0
    elif 'max' in df.loc[s,'pv_filename']:
        df.loc[s,'kwp_unit']= maximums[site]/df.loc[s,'number_of_households']
    else:
        df.loc[s,'kwp_unit'] = float(df.loc[s,'pv_filename'][-9]) + float(df.loc[s,'pv_filename'][-7])/10
    df.loc[s,'npv_unit'] = df.loc[s,'eno$_npv_building_mean']/df.loc[s,'number_of_households']
    bau_scenario = df[(df['site']==site) & (df['arrangement'] == 'bau') & (df['a_term'] == a_term)].index
    df.loc[s,'bau_npv'] = (df.loc[bau_scenario,'eno$_npv_building_mean']).values[0]/df.loc[s,'number_of_households']
    df.loc[s,'bau_scenario'] = bau_scenario
    df.loc[s,'npv_cf_bau'] = df.loc[s,'npv_unit'] - df.loc[s,'bau_npv']
    df.loc[s,'npv_%_bau'] = -df.loc[s,'npv_cf_bau'] / df.loc[s,'bau_npv'] * 100
    df.loc[s,'npv_cf_bau_000'] = df.loc[s,'npv_cf_bau'] /1000
df['parent_retail']= df.loc[df['parent'] != 'TIDNULL']['parent'].apply(lambda x: x[6:])
        

In [8]:
# Save updated results
fn = os.path.join(base_path, project,'outputs',study_name, study_name+'_results_process.csv')
df.to_csv(fn)